### Парсинг на Python:

Для начала я хочу написать, что я всего лишь 2 раза занимался такой штукой и сейчас решил изучить ее поподробнее. Для начала научусь работать с библиотекой BeatifulSoup и сделаю небольшую систему оповещений и она будет состоять из следующих тасков, которые будут обернуты в dag:
1. Паршу вакансии(практически закончил разобраться почему порой он берет значения не от туда, а также стоило бы добавить описание, чтобы сообщения отправлялись с пользой); +
2. Загружаю значения в бд; +
3. Беру только новые значения, а то есть left-only join; +
4. Отправляю все новые значения в телеграм канал каждый день, при этом также можно будет подумать в каком формате отсылать сообщения и в какие графики.

##### To-Do list:
1. Разобраться с выводом description, subway;
2. Оптимизировать парсер;
2. Подобрать оптимальные фильтры и написать таск;
3. Создать таск с ботом с отправкой сообщения.

In [1]:
#Для датафрейма:
import pandas as pd
import numpy as np

#Для визуализации:
import seaborn as sns
import matplotlib.pyplot as plt

#Для парсинга:
import requests
from hyper.contrib import HTTP20Adapter
from bs4 import BeautifulSoup as bs
from time import sleep

#Для airflow и clickhouse:
from airflow.decorators import dag, task
from datetime import datetime, timedelta
from clickhouse_driver import Client

#Для бота:
import telegram
import io

Сначала укажу все дефолтные параметры:

In [2]:
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=3),
    'start_date': datetime(2022, 9, 3),
}

In [3]:
client = Client(host='localhost')

In [4]:
my_token = "5711117341:AAEiU75eEX5RTbMEncnGh32JwsnhTSHtDrM"
bot = telegram.Bot(token=my_token)

In [5]:
updates = bot.getUpdates()
print(updates[-1])

{'channel_post': {'chat': {'id': -1001786025908, 'title': 'Квартиры в Москве', 'type': 'channel'}, 'channel_chat_created': False, 'photo': [], 'supergroup_chat_created': False, 'text': '/start', 'message_id': 4, 'date': 1662054739, 'sender_chat': {'id': -1001786025908, 'title': 'Квартиры в Москве', 'type': 'channel'}, 'delete_chat_photo': False, 'new_chat_members': [], 'new_chat_photo': [], 'caption_entities': [], 'group_chat_created': False, 'entities': [{'type': 'bot_command', 'length': 6, 'offset': 0}]}, 'update_id': 729828266}


In [8]:
chat_id = -1001786025908

In [9]:
schedule_interval = "00 22 * * 7"

Парсить я буду квартиры в Москве с сайта авито, далее я хочу поисследовать данные и сделать какие-то выводы.

Что буду парсить:
1. Ссылку;
2. Цену за квадратный метр;
3. Стоимость за квадратный метр;
4. Стоимость квартиры;
5. Описание;
6. Расстояние от метро.

In [53]:
lst_links = []
lst_square = []
lst_price = []
lst_subway = []
lst_description = []
lst_minutes = []

In [11]:
def parce_vacancies():
    #Сначала создаем цикл, где будем идти по страницам:
    for page in range(1, 99):
        print(page)  #чтобы отслеживать на какой страницы мы
        full_url = f"https://www.avito.ru/moskva/kvartiry/prodam-ASgBAgICAUSSA8YQ?cd=1&p={page}"
        source = requests.Session()
        source.mount('https://', HTTP20Adapter())
        response = source.get(full_url)
        sleep(7)  #чтобы не перегружался
        #инкодим значения иначе будет эльфийский
        response.encoding = 'utf-8'
        soup = bs(response.text, 'lxml')

        all_flats = soup.findAll("div", class_="iva-item-content-rejJg")

        #А здесь уже большой цикл обработки, где будем идти и собирать данные с каждой квартиры:
        for flat in all_flats:

            #Ссылка на квартиру:
            var_link = flat.find("a",
                                 "link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-listRedesign-_rejR title-root_maxHeight-X6PsH")
            if type(var_link) != type(None):
                local_var_link = "https://www.avito.ru" + var_link.get("href")
                lst_links.append([local_var_link])
            else:
                lst_links.append([None])

            #Отбираем цену за квадрат:
            var_square = flat.find("span",
                                   class_="price-noaccent-X6dOy price-normalizedPrice-PplY9 text-text-LurtD text-size-s-BxGpL")
            if type(var_square) != type(None):
                var = flat.find("span",
                                class_="price-noaccent-X6dOy price-normalizedPrice-PplY9 text-text-LurtD text-size-s-BxGpL").text
                correct_number_square_price = ""
                #Очищаем строку от бесполезного мусора:
                for var_square_circle in var:
                    if var_square_circle.isdigit():
                        correct_number_square_price += var_square_circle
                #Убираем последнею цифру, там значение в квадрате:
                correct_number_square_price = correct_number_square_price.replace(correct_number_square_price[-1], "")
                lst_square.append(correct_number_square_price)
            else:
                lst_square.append(None)

            #Отбираем полную цену:
            var_full_price = flat.find("span", class_="price-text-_YGDY text-text-LurtD text-size-s-BxGpL")
            if type(var_full_price) != type(None):
                full_price = var_full_price.text
                correct_number_full_price = ""
                #Оставляем лишь число:
                for var_full_price_circle in full_price:
                    if var_full_price_circle.isdigit():
                        correct_number_full_price += var_full_price_circle

                lst_price.append(correct_number_full_price)
            else:
                lst_price.append(None)

            #Отбираем станцию метро:
            var_subway = flat.find('div', class_="geo-georeferences-SEtee text-text-LurtD text-size-s-BxGpL")
            if type(var_subway) != type(None):
                subway_name = var_subway.text
                subway_full = ""
                for var_subway_circle in subway_name:
                    if var_subway_circle.isalpha():
                        subway_full += var_subway_circle
                    else:  #после станции идет время сколько от метро и это значение число, тем самым убираем его
                        break
                lst_subway.append(subway_full)
            else:
                lst_subway.append(None)

            #Описание квартиры:
            var_description = flat.find("div",class_="iva-item-text-Ge6dR iva-item-description-FDgK4 text-text-LurtD text-size-s-BxGpL")
            if type(var_description) != type(None):
                description = var_description.text
                lst_description.append(description)
            else:
                lst_description.append(None)

            #Добавляю расстояние от метро:
            var_minutes = flat.find("span",class_="geo-periodSection-bQIE4")
            if type(var_minutes) != type(None):
                var_minutes_text = var_minutes.text
                var_minutes_full = ""
                for var_minute_circle in var_minutes_text:
                    if var_minute_circle == '–' or var_minute_circle.isdigit():
                        var_minutes_full += var_minute_circle
                lst_minutes.append(var_minutes_full)
            else:
                lst_minutes.append(None)

    #Создаем датафрейм:
    full_df = pd.DataFrame(data={"square_price": lst_square, "full_price": lst_price, "subway": lst_subway,"links": lst_links,"description":lst_description,'destination_from_nearest_subway':lst_minutes})

    return full_df


In [12]:
full_df = parce_vacancies()

1
[2022-09-01 22:15:41,714] {connection.py:724} INFO - Received unhandled event <RemoteSettingsChanged changed_settings:{ChangedSetting(setting=SettingCodes.MAX_CONCURRENT_STREAMS, original_value=None, new_value=16), ChangedSetting(setting=SettingCodes.INITIAL_WINDOW_SIZE, original_value=65535, new_value=65536), ChangedSetting(setting=SettingCodes._max_frame_size, original_value=16384, new_value=16777215)}>
[2022-09-01 22:15:41,716] {connection.py:724} INFO - Received unhandled event <WindowUpdated stream_id:0, delta:2147418112>
[2022-09-01 22:15:41,738] {connection.py:724} INFO - Received unhandled event <SettingsAcknowledged changed_settings:{ChangedSetting(setting=SettingCodes.ENABLE_PUSH, original_value=1, new_value=0)}>
[2022-09-01 22:15:41,740] {connection.py:724} INFO - Received unhandled event <SettingsAcknowledged changed_settings:{}>
2
[2022-09-01 22:15:52,081] {connection.py:724} INFO - Received unhandled event <RemoteSettingsChanged changed_settings:{ChangedSetting(setting=

In [21]:
full_df_test = full_df.copy()

In [37]:
full_df_test.head()

,square_price,full_price,subway,links,description,destination_from_nearest_subway
0,370732,76000000,Баррикадная,https://www.avito.ru/moskva/kvartiry/6-k._kvar...,№044931.\n\nШестикомнатная квартира с отделкой...,11–15
1,228667,4116000,Лихоборы,https://www.avito.ru/moskva/kvartiry/apartamen...,Официальный Отдел Продаж.\n\nМного Разных Вари...,16–20
2,479616,40000000,Смоленская,https://www.avito.ru/moskva/kvartiry/2-k._kvar...,Лот №40434597. Исторический центр Москвы — Арб...,6–10
3,420635,26500000,Народное,https://www.avito.ru/moskva/kvartiry/2-k._kvar...,Лот №38198708. ЖК «Континенталь». \nПродается ...,21–30
4,743031,168668080,Охотный,https://www.avito.ru/moskva/kvartiry/3-k._kvar...,№117083.\n\nВ жилом комплексе «Дом Композиторо...,6–10


Создал sql запрос на создание таблички:

In [83]:
query = """
CREATE TABLE default.vacancies(
    square_price Int64,
    full_price Int64,
    subway String,
    links String,
    description String,
    destination_from_nearest_subway String)
ENGINE = MergeTree
Order by links
"""

Теперь создам таск небольшой пред обработкой:

In [32]:
full_df.isna().sum()

square_price    12
full_price       0
subway          20
links            0
dtype: int64

Значений совсем немного, так что предлагаю просто удалять строки, где есть Nan значение:

In [33]:
full_df.dropna(inplace=True)

In [35]:
full_df["links"] = full_df["links"].apply(lambda x: x[0])

In [39]:
full_df[["square_price","full_price"]] = full_df[["square_price","full_price"]].astype("int64")

In [41]:
full_df[["subway","links"]] = full_df[["subway","links"]].astype(str)

Напишу отдельную функция и в последствии таск для этого:

In [23]:
def filter_df(df):
    df.dropna(inplace=True)
    df["links"] = df["links"].apply(lambda x: x[0])
    df[["square_price","full_price"]] = df[["square_price","full_price"]].astype("int64")
    df[["subway","links","description","destination_from_nearest_subway"]] = df[["subway","links","description","destination_from_nearest_subway"]].astype(str)
    return df

In [24]:
full_df_test = filter_df(full_df_test)

Создам таск, в котором, буду отбирать только новые значения:

In [36]:
def uncommon_values(df_new):

    #Достаем все записи из таблицы и создаем новый датафрейм:
    df_old = client.execute("Select * from vacancies")
    df_old = pd.DataFrame(columns=["square_price","full_price","subway","links","description","destination_from_nearest_subway"], data=df_old)

    #Здесь выполняю left_only join для добавлении новых записей в таблицу и отправление файлика в тг:
    df = df_new.merge(df_old, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
    return df

In [46]:
uncommon_values(full_df_test)

,square_price,full_price,subway,links,description,destination_from_nearest_subway,_merge


Создам таск с добавлением новых записей в бд:

In [28]:
def to_clickhouse(df):
    client.execute("INSERT INTO default.vacancies VALUES", df.to_dict(orient="records"))
    #Строка не совсем важна, но при этом лучше ее оставить:
    client.execute('OPTIMIZE TABLE vacancies DEDUPLICATE BY links')

In [29]:
to_clickhouse(full_df_test)

In [48]:
full_df_test.subway.unique()

array(['Баррикадная', 'Лихоборы', 'Смоленская', 'Народное', 'Охотный',
       'Ховриноот', 'Крылатское', 'Строгинодо', 'Строгино',
       'Волгоградский', 'Петровско', 'Ботанический', 'Пятницкое', 'Юго',
       'Солнцево', 'Площадь', 'Проспект', 'Сходненская', 'Бабушкинская',
       'Авиамоторная', 'Окская', 'Тёплый', 'Беломорскаяот',
       'Домодедовская', 'Измайловская', 'Братиславская', 'Новогирееводо',
       'Свиблово', 'Перово', 'Мичуринский', 'Кузьминки', 'Ховрино',
       'Пролетарскаядо', 'Трубнаядо', 'Бабушкинскаяот', 'Марьина',
       'Молодёжная', 'Беломорская', 'Тропарёво', 'Селигерская',
       'Бутырская', 'Рассказовка', 'Пролетарская', 'Озёрная', 'ЗИЛ',
       'Римская', 'Минская', 'Бибирево', 'Медведковоот', 'Митино',
       'Алтуфьево', 'Ясенево', 'Пражская', 'Спортивнаядо', 'Октябрьское',
       'Рязанский', 'ВДНХ', 'Кожуховская', 'Бунинская', 'Новогиреево',
       'Новопеределкино', 'Тушинская', 'Нагатинскаяот', 'Шаболовская',
       'Тропарёводо', 'Селигерскаяот',

In [51]:
full_url = f"https://www.avito.ru/moskva/kvartiry/prodam-ASgBAgICAUSSA8YQ?cd=1&p=1"
source = requests.Session()
source.mount('https://', HTTP20Adapter())
response = source.get(full_url)
sleep(7)  #чтобы не перегружался
#инкодим значения иначе будет эльфийский
response.encoding = 'utf-8'
soup = bs(response.text, 'lxml')

all_flats = soup.findAll("div", class_="iva-item-content-rejJg")

[2022-09-01 22:55:18,850] {connection.py:724} INFO - Received unhandled event <RemoteSettingsChanged changed_settings:{ChangedSetting(setting=SettingCodes.MAX_CONCURRENT_STREAMS, original_value=None, new_value=16), ChangedSetting(setting=SettingCodes.INITIAL_WINDOW_SIZE, original_value=65535, new_value=65536), ChangedSetting(setting=SettingCodes._max_frame_size, original_value=16384, new_value=16777215)}>
[2022-09-01 22:55:18,851] {connection.py:724} INFO - Received unhandled event <WindowUpdated stream_id:0, delta:2147418112>
[2022-09-01 22:55:18,867] {connection.py:724} INFO - Received unhandled event <SettingsAcknowledged changed_settings:{ChangedSetting(setting=SettingCodes.ENABLE_PUSH, original_value=1, new_value=0)}>
[2022-09-01 22:55:18,868] {connection.py:724} INFO - Received unhandled event <SettingsAcknowledged changed_settings:{}>


In [66]:
lst_subway_test = []

In [67]:
for flat in all_flats:
    var_subway = flat.find('div', class_="geo-georeferences-SEtee text-text-LurtD text-size-s-BxGpL")
    if type(var_subway) != type(None):
        subway_name = var_subway.text
        print(subway_name)
        subway_full = ""
        for var_subway_circle in subway_name:
            if var_subway_circle.isalpha() or var_subway_circle == " " or var_subway_circle == "-":
                subway_full += var_subway_circle
            else:  #после станции идет время сколько от метро и это значение число, тем самым убираем его
                break
        lst_subway_test.append(subway_full)
    else:
        lst_subway_test.append(None)

Митино21–30 мин.
Площадь Ильича11–15 мин.
Пятницкое шоссеот 31 мин.
Говорово16–20 мин.
Мичуринский проспект16–20 мин.
Ховриноот 31 мин.
Стахановская6–10 мин.
Строгино6–10 мин.
Волгоградский проспект6–10 мин.
Строгино11–15 мин.
Волгоградский проспект6–10 мин.
Петровско-Разумовская6–10 мин.
Свиблово11–15 мин.
Проспект Вернадскогодо 5 мин.
Свиблово11–15 мин.
Рязанский проспект16–20 мин.
Пятницкое шоссеот 31 мин.
Юго-Западная16–20 мин.
Площадь Ильича16–20 мин.
Солнцево6–10 мин.
Волгоградский проспект6–10 мин.
Спартак21–30 мин.
Бабушкинская16–20 мин.
Волгоградский проспект6–10 мин.
Сходненская11–15 мин.
Тёплый стан16–20 мин.
Пятницкое шоссеот 31 мин.
Волгоградский проспект6–10 мин.
Домодедовская11–15 мин.
Измайловская6–10 мин.
Окская6–10 мин.
Площадь Ильича11–15 мин.
Алтуфьево6–10 мин.
Волгоградский проспект6–10 мин.
Сходненская11–15 мин.
р-н Молжаниновский
Волгоградский проспект11–15 мин.
Сходненская21–30 мин.
Площадь Ильича11–15 мин.
Молодёжная16–20 мин.
Площадь Ильича16–20 мин.
Проспект 

In [68]:
lst_subway_test

['Митино',
 'Площадь Ильича',
 'Пятницкое шоссеот',
 'Говорово',
 'Мичуринский проспект',
 'Ховриноот',
 'Стахановская',
 None,
 None,
 None,
 'Строгино',
 'Волгоградский проспект',
 'Строгино',
 'Волгоградский проспект',
 'Петровско-Разумовская',
 'Свиблово',
 'Проспект Вернадскогодо',
 'Свиблово',
 'Рязанский проспект',
 'Пятницкое шоссеот',
 'Юго-Западная',
 'Площадь Ильича',
 'Солнцево',
 'Волгоградский проспект',
 'Спартак',
 'Бабушкинская',
 'Волгоградский проспект',
 'Сходненская',
 None,
 'Тёплый стан',
 'Пятницкое шоссеот',
 'Волгоградский проспект',
 'Домодедовская',
 'Измайловская',
 'Окская',
 'Площадь Ильича',
 'Алтуфьево',
 'Волгоградский проспект',
 'Сходненская',
 'р-н Молжаниновский',
 'Волгоградский проспект',
 'Сходненская',
 'Площадь Ильича',
 'Молодёжная',
 'Площадь Ильича',
 'Проспект Вернадского',
 'Новогирееводо',
 'Сходненская',
 'Белорусская',
 'Ховрино',
 'Перово',
 'Мичуринский проспект',
 'Бульвар адмирала Ушакова']

In [94]:
lst_subway_test_new = []

In [101]:
for j in lst_subway_test:
    if j is not None and (j[-1] == "т" and j[-2] == "о") or j is not None and (j[-1] == "о" and j[-2] == "д"):


тесшекитП
тоивХ
ооосарВтеср
тесшекитП
тесшекитП
ооергвН


Ключевой момент здесь такой, что бывают название метро пишут и добавляют до и от.